In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.5 MB/s eta 0:00:00


In [ ]:
import nltk
import evaluate
import numpy as np
import pandas as pd

from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from datasets import Dataset,load_dataset

dataset = load_dataset("NebulaByte/E-Commerce_FAQs")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/659 [00:00<?, ? examples/s]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['parent_category', 'category_id', 'category', 'question_id', 'question', 'answer', 'faq_url', 'que_ans'],
        num_rows: 659
    })
})

In [ ]:
dataset = Dataset.from_dict({
    "question": [str(element) for element in dataset['train']['question']],
    "answer": [str(element) for element in dataset['train']['answer']]})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 527
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 132
    })
})

In [ ]:
prefix = "answer the question: "

def preprocess_function(examples):
    print(examples["answer"])
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/527 [00:00<?, ? examples/s]

['Individuals in the age group of 18-60 years can purchase standalone policies, provided he/she is not ineligible as per the terms and exclusions of the cover. Standalone policies for children below 18 years of age are not available.', 'Orders will be delivered by the date you see on the product page for your location. ', '\n\nYou can carry the date of birth certificate or passport for an infant. Vaccination cards are also accepted by a few airlines for domestic travel. However, you can get in touch with the airline for additional information.\n\n', "No, the refund for an order placed using the 'Credit Card No Cost EMI' payment mode can only be done to the credit card from which the payment was made.", 'Since payment modes cannot be changed after an order is placed, to cancel your EMI, your order will need to be cancelled.\n \n*Certain items cannot be cancelled after 24 hours of placing the order.', '\n\nBGauss vehicles must be serviced twice a year.\n\n', "For your 'Cash on Delivery o

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

['No, a spare tyre is not provided with Ather electric 2-wheeler.\n\xa0', 'Yes, you can delete your saved cards at any given time.', 'Regional Transport Office mein jo documents address proof ke taur pe jama dene hote hain veh state ke anusaar alag alag ho sakte hain. Haalanki, yeh saare documents address proof ke taur pe aksar maange jaate hain:\n- Aadhar Card- Passport- Bank Passbook - Voter ID- Electricity bill- LPG Connection bill- Water bill- Registered Rental Agreement', 'At Flipkart, all our sellers offer products from authorised vendors or brands directly and are 100% genuine. Rest assured, products will have the applicable warranty from brands as confirmed on the product page.', 'You can easily transfer the amount to your account from the Flipkart app or website.\nYou would not have to install the PhonePe app to transfer money from your PhonePe wallet to your bank account.', 'You can easily transfer the amount to your account from the Flipkart app or website. You would not hav

In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.184739,0.296347,0.132586,0.241927,0.256960
2,No log,2.106558,0.298995,0.145066,0.250394,0.265616


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=132, training_loss=2.247353524872751, metrics={'train_runtime': 104.145, 'train_samples_per_second': 10.121, 'train_steps_per_second': 1.267, 'total_flos': 50531615327232.0, 'train_loss': 2.247353524872751, 'epoch': 2.0})

In [ ]:
# question = "is there installation facality?"

# inputs = tokenizer.encode("answer the question:" + question, return_tensors="pt")

# output_ids = model.generate(inputs, max_length=1024, num_return_sequences=1)

# output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print("Generated Output:", output)

In [ ]:
import torch

ckpt_path = "ecom_qa_v1.ckpt"

torch.save(model.state_dict(), ckpt_path)

print("Model saved as checkpoint:", ckpt_path)


Model saved as checkpoint: ecom_qa_v1.ckpt


In [ ]:
import torch
from transformers import T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model1 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

ckpt_path = "ecom_qa_v1.ckpt"
model1.load_state_dict(torch.load(ckpt_path))

print("Model loaded from checkpoint:", ckpt_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded from checkpoint: ecom_qa_v1.ckpt


In [ ]:
def predict(question):
  inputs = tokenizer.encode("answer the question:" + question, return_tensors="pt").cuda()
  output_ids = model.generate(inputs, max_length=1024, num_return_sequences=1)
  output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  return output

In [ ]:
question = "What is the pickup process?"

print(predict(question))

The pickup process is the same as the delivery of the item. You can choose to pick it up from the seller's location or pick it up from the nearest UPS store.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ckpt_path = "/content/drive/My Drive/model/flipkart_qa_v1.ckpt"

# Save the model to the specified checkpoint file
torch.save(model1.state_dict(), ckpt_path)

print("Model saved as checkpoint in Google Drive:", ckpt_path)

Model saved as checkpoint in Google Drive: /content/drive/My Drive/model/flipkart_qa_v1.ckpt
